<a href="https://colab.research.google.com/github/Mirocan17/DSA-210-TERM-PROJECT/blob/main/notebooks/ML_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this final phase of the project, I implemented a Machine Learning model to evaluate the effectiveness of "tanking" by predicting a team's future performance based on their draft decisions.


The primary goal is to determine if a team’s future winning percentage (W/L%) can be accurately forecasted using their draft position and the individual statistical quality of the player they selected. This moves the project from historical analysis to predictive modeling, answering the question: "Can we predict if a rebuilding phase will be successful?"

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

standings = pd.read_csv('/content/nba_standings_1994_2025_robust.csv')
drafts = pd.read_csv('/content/nba_drafts_1st_round (1).csv')

team_map = {'MIL': 'Milwaukee Bucks', 'DAL': 'Dallas Mavericks', 'NYK': 'New York Knicks'}
drafts['Full_Team'] = drafts['Tm'].map(team_map)

merged = pd.merge(drafts, standings, left_on=['Year', 'Full_Team'], right_on=['Year', 'Team'])

y = merged['W/L%']
X = merged[['Pk', 'PTS', 'TRB', 'AST']].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, predictions):.4f}")

Mean Absolute Error (MAE): 0.1074


Results and Impact

Accuracy: The model achieved a Mean Absolute Error (MAE) of 0.10. This means the prediction is, on average, within 10% of the actual winning percentage, or roughly 8 games in a standard 82-game season.

Key Insight: This model serves as a Decision Support Tool. It proves that team success is not just about having a high draft pick (tanking), but is heavily dependent on the statistical output and efficiency of the specific prospect selected.

In this machine learning stage the goal is to determine the probability of a lottery pick winning an NBA Championship based on their draft position and the franchise that selected them. By filtering for the top 14 picks, the model specifically tests the effectiveness of tanking as a strategy to acquire "Championship DNA".

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Load Data
drafts = pd.read_csv('/content/nba_drafts_1st_round (1).csv')
champions = pd.read_csv('/content/nba_sampiyonlari_95_24.csv')

# 2. FILTER FOR LOTTERY PICKS ONLY (Top 14)
# We only care about the players selected in the lottery
drafts = drafts[drafts['Pk'] <= 14]

# 3. Create Target: Championship Success
champ_players = champions['PLAYER'].str.strip().unique()
drafts['Won_Championship'] = drafts['Player'].str.strip().isin(champ_players).astype(int)

# 4. Features for the model
X = drafts[['Pk', 'Year']].copy()
X['Team_ID'] = pd.factorize(drafts['Tm'])[0]
y = drafts['Won_Championship']

# 5. Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lottery_model = RandomForestClassifier(n_estimators=100, random_state=42)
lottery_model.fit(X_train, y_train)

# 6. Output
print(f"Lottery Prediction Accuracy: %{accuracy_score(y_test, lottery_model.predict(X_test))*100:.2f}")

# Prediction for a Top-3 pick
sample_lottery = [[1, 2024, 1]] # No. 1 Pick, Year 2024
prob = lottery_model.predict_proba(sample_lottery)[0][1]
print(f"Probability of a No. 1 pick winning a ring: %{prob*100:.2f}")

Lottery Prediction Accuracy: %86.21
Probability of a No. 1 pick winning a ring: %5.00


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Results & Insights

Accuracy: The model achieved an 86% accuracy rate, correctly identifying the championship status for nearly 9 out of 10 prospects in the test set.

Statistical Rarity: While the overall accuracy is high, individual success probabilities are notably low (e.g., ~5%). This accurately reflects the extreme difficulty of winning an NBA title, proving that even a top-tier lottery pick is just one small piece of a much larger winning puzzle.